# Análise do covid no Brasil

## Introdução

Esta é uma análise exploratória inicial dos dados do COVID-19 no Brasil.


### Objetivo

O principal objetivo é entender a distribuição da população contaminada por COVID-19 e verificar se fatores como idade, sexo e fatores de risco contribuem para complicação da doença.


### Metodologia

Na primeira parte, é feita uma análise na base e em suas variáveis, entendendo o significado e distribuição delas. E caso necessário, realizamos alguma limpeza/tratamento.

Na segunda parte da análise, criamos algumas hipóteses a respeito dos dados. 

### Resultado Esperado

O resultado sera analisado por meio de graficos, a fim de entendermos as distribuições e suas caracteristicas.
Além disso, podera ser usado um comparativo para ver se as hipóteses possuem significância estatística.


## Base de Dados

Foi utilizado dados abertos disponíveis em [https://opendatasus.saude.gov.br/dataset/bd-srag-2020](https://opendatasus.saude.gov.br/dataset/bd-srag-2020) com a data de 18/05/2020.

Esta base é disponibilizada pelo ***Vigilância de Síndrome Respiratória Aguda Grave (SRAG)***. O SRAG é um programa da Secretaria de Vigilância em Saúde (SVS).

O Dicionário com o significado de cada uma das colunas esta localizado [Aqui](https://opendatasus.saude.gov.br/dataset/9bc2013f-f293-4f3e-94e7-fa76204fc035/resource/20e51b77-b129-4fd5-84f6-e9428ab5e286/download/dicionario_de_dados_srag_hospitalizado_atual-sivepgripe.pdf)

## Análise da Base e das Características

In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [85]:
pd.options.display.max_columns = 20

In [86]:
df_full = pd.read_csv('data/sraghosp_18.05.2020.csv', low_memory=False)

In [87]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157268 entries, 0 to 157267
Columns: 141 entries, Unnamed: 0 to PAC_DSCBO
dtypes: float64(80), int64(11), object(50)
memory usage: 169.2+ MB


A base possui muitas colunas (141), e ao analisar o Dicionário de dados, notei que muitas delas não serão úteis para análise da população do COVID. Vamos ler o documento de Discionário e fazer uma filtragem em alto nível de quais colunas ficam.

In [88]:
cols_to_drop = ['Unnamed: 0', 'CS_ETINIA', 'NU_CEP', 'ID_PAIS', 'ID_RG_RESI', 'CO_RG_RESI', 'AVE_SUINO', 'NOSOCOMIAL', 'VACINA', 'DT_UT_DOSE', 'MAE_VAC', 'DT_VAC_MAE', 'M_AMAMENTA', 'DT_DOSEUNI', 'DT_1_DOSE', 'DT_2_DOSE', 'ANTIVIRAL', 'TP_ANTIVIR', 'OUT_ANTIV', 'DT_ANTIVIR', 'RAIOX_RES', 'RAIOX_OUT', 'DT_RAIOX', 'AMOSTRA', 'DT_COLETA', 'TP_AMOSTRA', 'OUT_AMOST', 'REQUI_GAL', 'DT_IF', 'POS_IF_FLU', 'TP_FLU_IF', 'POS_IF_OUT', 'IF_VSR', 'IF_PARA1', 'IF_PARA2', 'IF_PARA3', 'IF_ADENO', 'IF_OUTRO', 'DS_IF_OUT', 'LAB_IF', 'CO_LAB_IF', 'PCR_RESUL', 'DT_PCR', 'TP_FLU_PCR', 'PCR_FLUASU', 'FLUASU_OUT', 'PCR_FLUBLI', 'FLUBLI_OUT', 'POS_PCROUT', 'PCR_VSR', 'PCR_PARA1', 'PCR_PARA2', 'PCR_PARA3', 'PCR_PARA4', 'PCR_ADENO', 'PCR_METAP', 'PCR_BOCA', 'PCR_RINO', 'PCR_OUTRO', 'DS_PCR_OUT', 'CLASSI_OUT', 'HISTO_VGM', 'PAIS_VGM', 'CO_PS_VGM', 'LO_PS_VGM', 'DT_VGM', 'DT_RT_VGM', 'PCR_SARS2', 'PAC_COCBO', 'PAC_DSCBO',]
df_clean = df_full.drop(cols_to_drop, axis=1)

In [89]:
df_clean.info(verbose=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 157268 entries, 0 to 157267
Columns: 71 entries, DT_NOTIFIC to DT_DIGITA
dtypes: float64(40), int64(9), object(22)
memory usage: 85.2+ MB


In [90]:
df_clean.sample(8)

,DT_NOTIFIC,SEM_NOT,DT_SIN_PRI,SEM_PRI,SG_UF_NOT,ID_REGIONA,CO_REGIONA,ID_MUNICIP,CO_MUN_NOT,ID_UNIDADE,...,IF_RESUL,POS_PCRFLU,LAB_PCR,CO_LAB_PCR,CLASSI_FIN,CRITERIO,EVOLUCAO,DT_EVOLUCA,DT_ENCERRA,DT_DIGITA
22904,2020-03-18,12,2020-03-16,12,DF,NaN,NaN,BRASILIA,530010,HRL,...,4.0,2.0,LACEN DF,11371.0,2.0,1.0,1.0,2020-03-26,2020-03-27,2020-03-26
75583,2020-04-19,17,2020-04-12,16,SP,GVE I CAPITAL,1331.0,SAO PAULO,355030,UPA PIRITUBA,...,4.0,2.0,INSTITUTO ADOLFO LUTZ SAO PAULO,2091364.0,5.0,1.0,1.0,2020-05-04,2020-05-07,2020-04-29
123461,2020-04-28,18,2020-04-27,18,RJ,NaN,NaN,RIO DE JANEIRO,330455,PERINATAL BARRA CASA DE SAUDE LARANJEIRAS,...,5.0,NaN,NaN,NaN,5.0,1.0,NaN,NaN,NaN,2020-05-13
100110,2020-04-28,18,2020-04-20,17,AM,ENTORNO DE MANAUS E RIO NEGRO,5584.0,MANAUS,130260,HOSPITAL P S DR JOAO LUCIO P MACHADO,...,5.0,NaN,LACEN LABORATORIO CENTRAL DE SAUDE PUBLICA DO ...,2018764.0,5.0,1.0,2.0,2020-04-28,2020-05-04,2020-04-29
124484,2020-05-01,18,2020-04-28,18,BA,NUCLEO REGIONAL DE SAUDE LESTE,1380.0,SALVADOR,292740,HOSPITAL SAO RAFAEL,...,4.0,NaN,HOSPITAL SAO RAFAEL,3808.0,5.0,1.0,1.0,2020-05-03,2020-05-08,2020-05-01
63173,2020-04-16,16,2020-04-11,15,SP,GVE I CAPITAL,1331.0,SAO PAULO,355030,HOSPITAL SAO CAMILO IPIRANGA,...,4.0,NaN,HOSPITAL ISRAELITA ALBERT EINSTEIN,2058391.0,5.0,1.0,2.0,2020-05-16,2020-05-18,2020-05-18
75790,2020-04-27,18,2020-04-13,16,PE,001,1497.0,RECIFE,261160,PROCAPE,...,5.0,NaN,NaN,NaN,5.0,1.0,2.0,2020-04-27,2020-05-16,2020-05-08
105844,2020-05-01,18,2020-04-25,17,PE,001,1497.0,RECIFE,261160,HOSPITAL GETULIO VARGAS,...,5.0,NaN,LAB CENTRAL DE SAUDE PUB DR MILTON BEZERRA SOBRAL,2712075.0,5.0,1.0,NaN,NaN,NaN,2020-05-13


In [96]:
df = df_clean

In [91]:
sintomas_cols = ['FEBRE', 'TOSSE', 'GARGANTA', 'DISPNEIA', 'DESC_RESP', 'SATURACAO', 'DIARREIA', 'VOMITO', 'OUTRO_DES']
# OUTRO_DES if OUTRO_SIN == 1

In [105]:
fatores_risco_cols = ['PUERPERA', 'CARDIOPATI', 'HEMATOLOGI', 'SIND_DOWN', 'HEPATICA', 'ASMA', 'DIABETES', 'NEUROLOGIC', 'PNEUMOPATI', 'IMUNODEPRE', 'RENAL', 'OBESIDADE', 'MORB_DESC']
# MORB_DESC if OUT_MORBI == 1
# if OBESIDADE==1 then OBES_IMC is not null
df[(df.CARDIOPATI==1) & (df.EVOLUCAO==1)].EVOLUCAO.count()

4080

In [97]:
df[df.CLASSI_FIN==5].shape

(43906, 71)

In [98]:
## CLASSI_FIN==5 is for COVID only
df = df[df_full.CLASSI_FIN==5]
df.groupby('UTI').UTI.count()

UTI
1.0    13848
2.0    22361
9.0     1030
Name: UTI, dtype: int64